In [1]:
!pip install opentree

In [2]:
from opentree import OT

Get the list of taxa from a textfile.  

In [3]:
fi = open("CALeDNAformatTwierdattempt.txt").readlines()

In [4]:
fi[0].split(';')

['Eukaryota',
 'Chlorophyta',
 'Ulvophyceae',
 'Bryopsidales',
 'Codiaceae',
 'Codium',
 'Codium fragile subsp. tomentosoides\n']

In [6]:
fi[0].split(';')[4]#[1:4]

'Codiaceae'

In [9]:
all_genera = set()
for taxon in fi:
    try:
        genus = taxon.split(';')
        print(OT.get_ottid_from_name(genus))[4]
        all_genera.add(genus)
    except:
        pass
        

In [8]:
len(all_genera)

0

In [14]:
OT.get_ottid_from_name('Sprivivirus')

IndexError: list index out of range

In [10]:
def get_matchdict_from_taxlist(list_of_taxa):
        """Returns tree object
        """
        matches = dict()
        failed = set()
        for tax in list_of_taxa:
            tax = tax.strip()
            print(tax)
            if tax != '':
                ott_id = OT.get_ottid_from_name(genus)
                print(ott_id)
                matches[genus] = ott_id

In [11]:
#all_genera_test = set()
#all_genera_test.add("Maeotias")
#type(all_genera_test)
#print(all_genera_test)
#print(all_genera)
#get_matchdict_from_taxlist(all_genera_test)

In [11]:
get_matchdict_from_taxlist(all_genera)

NameError: name 'length' is not defined

In [ ]:
matches = dict()
ottid_to_genus = dict()
for genus in all_genera:
    if len(genus) > 2:
        #try:
            ott_id = OT.get_ottid_from_name(genus)
            ottid_to_genus['ott{}'.format(ott_id)]=genus
            matches[genus] = ott_id
        #except:
        #    print("Failed to get an ottid for {}".format(genus))

In [13]:
len(all_genera)

519

In [ ]:
output = OT.synth_induced_tree(ott_ids=list(matches.values()),  label_format='name_and_id')

In [ ]:
#Create a dictionary that matches the taxa that are returned with the input taxa requested.
relabel = dict()
broken = output.response_dict['broken']
for taxon in broken:
    remap = broken[taxon]
    if remap.startswith('mrca'):
        if remap not in relabel:
            relabel[remap] = []
        relabel[remap].append("{} {}".format(ottid_to_genus[taxon], taxon))
    if remap.startswith('ott'):
        if remap not in relabel:
            relabel[remap] = []
        relabel[remap].append("{} {}".format(ottid_to_genus[taxon], taxon))

In [ ]:
import copy
backuptree = copy.deepcopy(output.tree)

In [ ]:
#Gos through each taxon in teh tree that is returned, and matches it to the rquested genus or genera, and adds those to the label
for taxon in backuptree.taxon_namespace:
    if taxon.label.startswith('mrca'):
        taxon.label = 'MRCA of taxa in '+' '.join(relabel[taxon.label])
    else:
        ott = taxon.label.split()[-1]
        if ott in relabel:
            added_taxa = ' MRCA of taxa in '+' '.join(relabel[ott])
            taxon.label = taxon.label + added_taxa

In [ ]:
# Some characters in names mess up newick tree readers
def remove_problem_characters(instr, prob_char = "():#", replace_w = '?'):
    problem_characters = set(prob_char)
    for char in problem_characters:
        instr = instr.replace(char,replace_w)
    return instr

In [ ]:
#Some of these taxon names are still causing issues...
for taxon in backuptree.taxon_namespace:
    taxon.label = remove_problem_characters(taxon.label)

for node in backuptree:
    if node.label:
        node.label = remove_problem_characters(node.label)

In [ ]:
backuptree.print_plot(width=100)
treefile = "bigbio_clean.tre"
backuptree.write(path = treefile, schema = "newick")

In [ ]:
studies = output.response_dict['supporting_studies']
cites = OT.get_citations(studies) #this can be a bit slow
print(cites)